# HW 1: Corporate Bond Pricing (due by 9.17 Tues)

We are going to compute the price of a corporate bond (subject to default) with Monte-Carlo simulation. Assume that 
* the default time of a company follows the exponential distribution with intensity $\lambda=$__`def_rate`__. 
* the riskfree interest rate is $r_f=$__`rf_rate`__ and the maturity of the bond is $T=$__`mat`__. 
* in the case of default, you can recover some portion ($R=$__`recovery_rate`__) of the face value.
* the coupon is 0%, i.e., it is a zero-coupon bond.
* the face value of the bond is 1.0
* use compound rate for discounting; the price of the default-free bond is $e^{-r_f T}$

The Problem 1 of the [2017 ASP Midterm Exam](../files/ASP2017_Midterm.pdf) will be helpful.

### Instruction to upload your HW
* Create a repository named __`PHBS_ASP_2019`__ (and clone it to your PC)
* Copy this file to __`PHBS_ASP_2019/HW1/HW1.ipynb`__  (Please use the same name for repository and ipynb file)
* Add solution code.
* Run your your code to make sure that there's no error.
* Upload (commit and sync) your file.

### 1. First, let's create a pricing function and check the std 

In [1]:
import numpy as np

In [2]:
def_rate = 0.1
rf_rate = 0.03
recovery = 0.3
mat = 10

In [3]:
# First generate exponential random numbers
# Although you can generate directly using fault_time = np.random.exponential(scale=), let's use uniform random numbers.
n_sample = 10000
U = np.random.uniform(size=n_sample)
default_time = -(1/def_rate)*np.log(U)

# You can check if the RNs are correct by comparing the means
(default_time.mean(), 1/def_rate)

(9.965145820782153, 10.0)

In [4]:
# Put your code here to price the corporate bond

def corp_bond(mat=1, def_rate=0.03, rf_rate=0.04, recovery=0.3, n_sample=1e4):
    n_sample=int(n_sample)
    U = np.random.uniform(size=n_sample)
    default_time = -(1/def_rate)*np.log(U)
    CBPriceSample=[]
    for i in range(0,int(n_sample)):
        if default_time[i]<=mat:
            CBPriceSample.append(recovery*np.exp(-rf_rate*default_time[i]))
        else:
            CBPriceSample.append(np.exp(-rf_rate*mat))
    return np.mean(np.array(CBPriceSample))

# Call your function
print(corp_bond(mat, def_rate, rf_rate, recovery, n_sample))

# Find the mean and std by calling the function 100 times. 

MCRecord=[]
for i in range(0,100):
    MCRecord.append(corp_bond(mat, def_rate, rf_rate, recovery, n_sample))
MCmean=np.mean(np.array(MCRecord))
MCStd=np.std(np.array(MCRecord))
print(MCmean)
print(MCStd)

0.44153192751096204
0.44047643947611803
0.00233110622838855


### 2. Now, let's improve the function by reducing the MC variations.
1. Use antithetic method: If `U` is uniform random variable, so is `1-U`
2. Also shift the RNs to match the mean, `1/def_rate`

In [5]:
# For example, antithetic method mean
n_sample = 10000
U = np.random.uniform(size=int(n_sample/2))
default_time = -(1/def_rate)*np.log(np.concatenate((U,1-U),axis=0))

# Mean-matching means
default_time += 1/def_rate-default_time.mean()
(default_time.mean(), 1/def_rate)

(10.0, 10.0)

In [6]:
# No include the two new features: `antithetic` and `mean_match`

def corp_bond_cv(mat=1, def_rate=0.03, rf_rate=0.04, recovery=0.3, n_sample=1e4, antithetic=True, mean_match=True):
    ### <--
    if(antithetic):
        U = np.random.uniform(size=int(n_sample/2))
        default_time = -(1/def_rate)*np.log(np.concatenate((U,1-U),axis=0))
    else:
        U = np.random.uniform(size=n_sample)
        default_time = -(1/def_rate)*np.log(U)
        
    if(mean_match):
        default_time += 1/def_rate-default_time.mean()
    
    CBPriceSample=[]
    for i in range(0,int(n_sample)):
        if default_time[i]<=mat:
            CBPriceSample.append(recovery*np.exp(-rf_rate*default_time[i]))
        else:
            CBPriceSample.append(np.exp(-rf_rate*mat))
    return np.mean(np.array(CBPriceSample))

# Find the mean and std by calling the function 100 times for (i) antithetic (ii) mean_match and (iii) both
MCRecordAnti=[]
for i in range(0,100):
    MCRecordAnti.append(corp_bond_cv(mat, def_rate, rf_rate, recovery, n_sample,True,False))
MCmeanAnti=np.mean(np.array(MCRecordAnti))
MCStdAnti=np.std(np.array(MCRecordAnti))
print("Antithetic Method")
print(MCmeanAnti)
print(MCStdAnti)

MCRecordMatch=[]
for i in range(0,100):
    MCRecordMatch.append(corp_bond_cv(mat, def_rate, rf_rate, recovery, n_sample,False,True))
MCmeanMatch=np.mean(np.array(MCRecordMatch))
MCStdMatch=np.std(np.array(MCRecordMatch))
print("Mean-Matching Method")
print(MCmeanMatch)
print(MCStdMatch)

MCRecordBoth=[]
for i in range(0,100):
    MCRecordBoth.append(corp_bond_cv(mat, def_rate, rf_rate, recovery, n_sample,True,True))
MCmeanBoth=np.mean(np.array(MCRecordBoth))
MCStdBoth=np.std(np.array(MCRecordBoth))
print("Both Methods")
print(MCmeanBoth)
print(MCStdBoth)

Antithetic Method
0.44067361621452555
0.0015855891565306572
Mean-Matching Method
0.44050752561509937
0.0015009565836833821
Both Methods
0.4404578412267239
0.0014298788608125177


### 3. Finally, what is the analytic value of the corporate bond? How does it compare to your MC result above?

In [7]:
### Put the analytic expression for the corporate bond price

r = def_rate + rf_rate
price_analytic = 0.3 * (1 - np.exp(-r*mat)) * def_rate / r + np.exp(-r * mat)
print("Analytic Expression\n",price_analytic)

Analytic Expression
 0.44040907156462505
